# The commands `grep` and `awk`

In this section we will introduce the commands `grep` and `awk`.

## Searching inside files with `grep`

A common task is to extract information from large files. This can be achieved using the Linux command `grep`, which stands for "Globally search for a Regular Expression and Print". The meaning of this acronym will become clear later, when we discuss Regular Expressions. First, we will consider simpler examples.

Before we start, change into the `grep` directory:

In [ ]:
cd ../grep

### Simple pattern matching

We will use a small example file (in "BED" format), which contains information about gene features in a genome. A bed file is a column-based file, each column contains information about the feature and is seperated by a tab character. There can be more than 10 columns, but only the first three are required to be a valid file. The file format is described in full here: [http://genome.ucsc.edu/FAQ/FAQformat#format1](http://genome.ucsc.edu/FAQ/FAQformat#format1). We will use the first 5 columns:

1. Sequence name
2. start position (starting from 0, not 1)
3. end position (starting from 0, not 1)
4. feature name
5. score (which is used to store the gene expression level in our examples).

Here is the contents of the first example BED file used in this section:

In [ ]:
cat gene_expression.bed

In reality, such a file could contain 100,000s of lines, so that it is not practical to read manually. Suppose we are interested in all the genes from chromosome 2. We can find all these lines using grep:

In [ ]:
grep chr2 gene_expression.bed

This has shown us all the lines that contain the text or string "chr2".

We can use a pipe to then just extract the genes that are on the positive strand, using grep a second time:

In [ ]:
grep chr2 gene_expression.bed | grep +

However, since `grep` is reporting a match to a string *anywhere* on a line, such simple searches can have undesired consequences. For example, consider the result of doing a similar search for all the genes in chromosome 1:




In [ ]:
grep chr1 gene_expression.bed

Oops! We found genes in chromosome 10, because "chr1" is a substring (subset) of "chr10".

Or consider the following file, where the genes have unpredictable names (which is not unusual for bioinformatics data).

In [ ]:
cat gene_expression_sneaky.bed

 Now we try to find genes on chromosome 1 that are on the negative strand. We put the minus sign in quotes, to stop Linux interpreting this as an option to `grep`, as opposed to the string we are searching for:

In [ ]:
grep chr1 gene_expression_sneaky.bed | grep '-'

The extra lines are found by `grep` because of matches in columns we were not expecting to match. Remember, `grep` is reporting these lines because they each contain the strings "chr1" and "-" *somewhere*.

We need a way to make searching with `grep` more specific.

### Regular expressions

Regular expressions provide the solution to the above problems. They are ways of defining more specific patterns to search for.

#### Matching the start and end of lines

First, we can specify that a match must be at the start of a line using the symbol "`^`", which means "start of line". Without the `^`, we find any match to "chr1":

In [ ]:
grep chr1 gene_expression_sneaky.bed

In [ ]:
grep '^chr1' gene_expression_sneaky.bed

Good! We have removed the match to the badly-named gene "chr11.gene1", which is on chromosome 8. Now we want to avoid matching chromosomes 10 and 11. This can be done by also looking for a "tab" character, which is represented by writing `\t`. For technical reasons, which are beyond the scope of this course, we must also put a dollar sign before the quotes to make any search involving a tab character work.

In [ ]:
grep $'^chr1\t' gene_expression_sneaky.bed

To find the genes on the negative strand, all that remains is to match a minus sign at the *end* of the line (so that we do not find "sneaky-gene3"). We can do this using the dollar "`$`", which means "end of line".

In [ ]:
grep $'^chr1\t' gene_expression_sneaky.bed | grep '\-$'

#### Wildcards and alphabets

Another special character in regular expressions is the dot: ".". This stands for any single character. For example, this finds all matches to chromosomes 1-9, and chromosomes X and Y:


In [ ]:
grep $'^chr.\t' gene_expression.bed

In fact, the earlier command that found all genes on chromosome 1 that are on the negative strand, could be found with a single call to `grep` instead of two calls piped together. To do this, we need a regular expression that finds lines that:

* start with chr1, then a tab character
* end with a minus
* have arbitrary characters between.

The asterisk "\*" has a special meaning: it says to match any number (including zero) of whatever character is before the \*. For example, the regular expression 'AC\*G' will match AG, ACG, ACCG, etc. The simpler, improved command is:

In [ ]:
grep $'^chr1\t.*-$' gene_expression_sneaky.bed

As well as matching any character using a dot, we can define any list of characters to match, using square brackets. For example, [12X] means match a 1, 2, or an X. This can be used to find all genes from chromosomes 1, 2 and X: 

In [ ]:
grep $'^chr[12X]\t' gene_expression.bed

Or just the autosomes may be of interest. To do this we introduce two new features:

* Ranges can be given in square brackets, for example [1-5] will match 1, 2, 3, 4 or 5.
* The plus sign "+" has a special meaning that is similar to "\*". Instead of any number of matches (including zero), it looks for at least one match. To avoid simply matching a plus sign, it must be preceded by a backslash: "\\+". For example, the regular expression 'AC\\+G' will match ACG, ACCG, ACCCG etc (but will not match AG).

Warning: Adding a backslash is often called _escaping_ (e.g. _escape the plus symbol_).  Depending on the software you're using (and the options you give it), you may need to escape the symbol to indicate that you want its special regex meaning (e.g. multiple copies of the last character please) or its literal meaning (e.g. give me a '+' symbol please).  If your command isn't working as you expect, try playing with these options and always test your regular expression before assuming it gave you the right answer.

The command to find the autosomes is:

In [ ]:
grep $'^chr[0-9]\+\t' gene_expression.bed

### Other `grep` options
The Linux command `grep` and regular expressions are extremely powerful and we have only scratched the surface of what they can do. Take a look at the manual (by typing `man grep`) to get an idea. A few particularly useful options are discussed below.

In [ ]:
man grep

#### Counting matches
A common scenario is counting matches within files. Instead of output each matching line, the option "`-c`" tells `grep` to report the number of lines that matched. For example, the number of genes in the autosomes in the above example can be found by simply adding `-c` to the command.

In [ ]:
grep -c $'^chr[0-9]\+\t' gene_expression.bed

#### Case sensitivity
By default, `grep` is case-sensitive. It can be useful to ignore the distinction between upper and lower case using the option "`-i`". Suppose we have a file of sequences, and want to find the sequences that contain the string ACGT. It is not unusual to come across files that have a mix of upper and lower case nucleotides. Consider this FASTA file:

In [ ]:
cat sequences.fasta

A simple search for ACGT will not return all the results:

In [ ]:
grep ACGT sequences.fasta

However, making the search case-insensitive solves the problem.

In [ ]:
grep -i ACGT sequences.fasta

#### Inverting matches
By default, `grep` reports all lines that do match the regular expression. Sometimes it is useful to filter a file, by reporting lines that *do not* match the regular expression. Using the option "`-v`" makes `grep` "invert" the output. For example, we could exclude genes from autosomes in the BED file from earlier.

In [ ]:
grep -v $'^chr[0-9]\+\t' gene_expression.bed

### Replacing matches to regular expressions
Finally, we show how to replace every match to a regular expression with something else, using the command "`sed`". The general form of this is:

    sed 's/regular expression/new string/' input_file
    
This will output a new version of the input file, with each match to the regular expression replaced with "`new string`". For example, try:

In [ ]:
sed 's/^chr/chromosome/' gene_expression.bed

This will replace every occurence of the text "chr" that appears at the start of a line in the file ``gene_expression.bed`` with the text "chromosome".

### Exercises

The following exercises all use the FASTA file `exercises.fasta`. Before starting the exercises, open a new terminal and navigate to the `linux/data/grep` directory, which contains `exercises.fasta`.

Use `grep` to find the answers. Hint: some questions require you to use `grep` twice, and possibly some other Linux commands.

1. Make a `grep` command that outputs just the lines with the sequence names.
2. How many sequences are in the file?
3. Do any sequence names have spaces in them? What are their names?
4. Make a `grep` command that outputs just the lines with the sequences, not the names.
5. How many sequences contain unknown bases (an "n" or "N")?
6. Are there any sequences that contain non-nucleotides (something other than A, C, G, T or N)?
7. How many sequences contain the 5' cut site GCWGC (where W can be an A or T) for the restriction enzyme AceI?
8. Are there any sequences that have the same name? You do not need to find the actual repeated names, just whether any names are repeated. (Hint: it may be easier to first discover how many unique names there are).

## File processing with AWK

AWK is a programming language named after the initials of its three inventors: Alfred **A**ho, Peter **W**einberger, and Brian **K**ernighan. AWK is incredibly powerful at processing files, particularly column-based files, which are commonplace in Bioinformatics. For example, BED, GFF, and SAM files.

Although long programs, put into a separate file, can be written using AWK, we will use it directly on the command line. Effectively, these are very short AWK programs, often called "one-liners".

Before we start, change into the `awk` directory:

In [ ]:
cd ../awk

### Extracting columns from files
The command `awk` reads a file line-by-line, splitting each line into columns. This makes it easy to do simple things like extract a column from a file. We will use the following GFF file for our examples.

In [ ]:
cat genes.gff

A GFF file is similar to a BED file in that it contains information about gene features in a genome. It is a column-based file, each column contains information about the feature. The columns in the GFF file are separated by tabs and each column has the following meaning:

1. Sequence name
2. Source - the name of the program that made the feature
3. Feature - the type of feature, for example gene or CDS
4. Start position
5. Stop position
6. Score
7. Strand (+ or -)
8. Frame (0, 1, or 2)
9. Optional extra information, in the form key1=value1;key2=value2;...

The score, strand, and frame can be set to '.' if it is not relevant for that feature. The final column 9 may or may not be present and could contain any number of key, value pairs.

We can use `awk` to just print the first column of the file. `awk` calls the columns `$1`, `$2`, ... etc, and the complete line is called `$0`. Try

In [ ]:
awk -F"\t" '{print $1}' genes.gff

A little explanation is needed.

* The option `-F"\t"` was needed to tell `awk` that the columns are separated by tabs (more on this later).
* For each line of the file, `awk` does what is inside the curly brackets. In this case, we simply print the first column ($1).

The repeated chromosome names are not nice. It is more likely to want to know just the unique names, which can be found by piping into the Linux command `sort -u`. 

In [ ]:
awk -F"\t" '{print $1}' genes.gff | sort -u

In this command the `|` symbol is known as the _pipe_ symbol. This _pipes_ (sends) the output of the `awk` command into the input of `sort -u`. The `sort` will sort the contents of the input. When you sort the input, lines with identical content end up next to each other in the output. The `-u` option of the sort command will select only the unique values in the output.

You can connect as many commands as you want. For example, type:

In [ ]:
awk -F"\t" '{print $1}' genes.gff | sort -u | wc -l

This will count the number of lines in the output.

### Filtering the input file
Similarly to `grep`, `awk` can be used to filter out lines of a file. However, since awk is column-based, it makes it easy to filter based on properties of any columns of interest. The filtering criteria can be added before the braces. For example, the following extracts just chromosome 1 from the file.

In [ ]:
awk -F"\t" '$1=="chr1" {print $0}' genes.gff

There are two important things to note from the above command:

1. `$1=="chr1"` means that column 1 must be *exactly* equal to "chr1". This means that "chr10" is not found.
2. The "`{print $0}`" part only happens when the first column is equal to "chr1", otherwise `awk` does nothing (the line gets ignored).

Awk commands are made up of two parts, a _pattern_ (e.g. `$1=="chr1"`) and an _action_ (e.g. `print $0`) which is contained in curly braces.  The _pattern_ defines which lines the _action_ is applied to.

In fact, the action (the part in curly braces) can be omitted in this example. `awk` assumes that you want to print the whole line, unless it is told otherwise. This gives a simple method of filtering based on columns.

### Exercises

The following exercises all use the BED file `exercises.bed`. Before starting the exercises, open a new terminal and navigate to the `awk` directory, which contains `exercises.bed`.

Use `awk` to find the answers to the following questions about the file `exercises.bed`. Many questions will require using pipes (eg "`awk ... | sort -u`" for question 1).

1. What are the names of the contigs in the file?
2. How many contigs are there?
3. How many features are on the positive strand?
4. How many features are on the negative strand?
5. How many repeat features are there?

When you have completed these exercises move on to the next part of the tutorial, [Advanced Linux](advanced_linux.ipynb).